In [1]:
import math
import torch
import numpy as np
import gpytorch
import pandas as pd
from matplotlib import pyplot as plt
import random
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

In [3]:
#normalizing the inputs
xmeans = x_pd.mean(axis=0)
xstddv = x_pd.std(axis=0)
x_pd_normal = (x_pd - xmeans)/xstddv

x_pd = x_pd_normal

x_mins = x_pd.min(axis=0).to_numpy()
x_maxs = x_pd.max(axis=0).to_numpy()

In [4]:
#normalize objective value data
y_obj_pd = y_pd.iloc[:, [0,1,2]]
ymeans = y_obj_pd.mean(axis=0)
ystddv = y_obj_pd.std(axis=0)
y_obj_pd_normal = (y_obj_pd - ymeans) / ystddv

y_pd.iloc[:, [0,1,2]] = y_obj_pd_normal

In [5]:
objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']
# objective_properties = ['Polymer Solubility', 'Gelation Enthalpy']
scaling = {'ymeans': ymeans[objective_properties],
           'ystddv': ystddv[objective_properties],
           'xmeans': xmeans,
           'xstddv': xstddv}

In [6]:
validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

In [7]:
#make torch tensors
train_x = torch.tensor(train_x_pd.values, dtype=torch.float)
train_y1 = torch.tensor(train_y_pd['Polymer Solubility', 'mg/mL'].values, dtype=torch.float).squeeze()
train_y2 = torch.tensor(train_y_pd['Gelation Enthalpy', 'J/g'].values, dtype=torch.float).squeeze()
train_y3 = torch.tensor(train_y_pd['Shear Modulus', 'Kpa'].values, dtype=torch.float).squeeze()
train_y4 = torch.tensor(train_y_pd['Manufacturability', '--'].values, dtype=torch.long).squeeze()

In [8]:
test_x = torch.tensor(x_pd.values, dtype=torch.float)
test_y = torch.tensor(y_pd.values, dtype=torch.float).squeeze()

In [9]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# function to optimize parameters of the regression GP -
def train_reg_gp(model, likelihood, train_x, train_y, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if i - 1  == training_iter:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()

    return model, likelihood 

In [10]:
class DirichletGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# function to optimize parameters of the classification GP - 
def train_cls_gp(model, likelihood, train_x, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
        optimizer.step()

    return model, likelihood


In [11]:
#define likelihood and initialize model - regression
reg_likl = gpytorch.likelihoods.GaussianLikelihood()
reg_model1 = ExactGPModel(train_x, train_y1, reg_likl)
reg_model2 = ExactGPModel(train_x, train_y2, reg_likl)
reg_model3 = ExactGPModel(train_x, train_y3, reg_likl)

#define likelihood and initialize model - classification
cls_likl = gpytorch.likelihoods.DirichletClassificationLikelihood(train_y4, learn_additional_noise=False, alpha_epsilon=1e-4)
cls_model = DirichletGPModel(train_x, cls_likl.transformed_targets, cls_likl, num_classes=cls_likl.num_classes)

In [12]:
training_iter = 50

#train model - regression
reg_model1, reg_likl1 = train_reg_gp(reg_model1, reg_likl, train_x, train_y1, training_iter)
reg_model2, reg_likl2 = train_reg_gp(reg_model2, reg_likl, train_x, train_y2, training_iter)
reg_model3, reg_likl3 = train_reg_gp(reg_model3, reg_likl, train_x, train_y3, training_iter)

#train model - regression
cls_model, cls_likl = train_cls_gp(cls_model, cls_likl, train_x, training_iter)

In [13]:
def predict(model, likl, X):
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = likl(model(X))
        mean = pred.mean
        std = pred.stddev
    return mean, std

#sum the available models to convert mutli obj problem to single obj
def multi_objective_predict(X, models, liklihoods):
    mean = 0
    var = 0
    for i in range(len(models)):
        tmp_mean, tmp_std = predict(models[i], liklihoods[i], torch.tensor(X, dtype=torch.float))
        mean += tmp_mean
        var += tmp_std**2
    return mean, var**0.5

# function to calculate acquisition function - feasibility weighted acquisition
def acquisition_func(X, models, liklihoods, y_min_curr):
    X = np.reshape(X, (1, -1))
    pred_mean, pred_std = multi_objective_predict(X, models[0:-1], liklihoods[0:-1]) #get prediction mean and standard dev
    cls_probs = class_probability(X, models[-1]) # vector with class probabilities prediction
    feasible_probs = cls_probs[1] #class 1 is assumed as feasible class

    improv = pred_mean - y_min_curr
    z_score = np.divide(improv, pred_std + 1E-9)
    acf = np.multiply(improv, norm.cdf(z_score)) + np.multiply(pred_std, norm.pdf(z_score))
    return (-1.0) * acf * feasible_probs        # weighting acf value with Pr(feasible|X)

#function to calculate class probabilities
def class_probability(X, model):
    with torch.no_grad():
        logit_dist = model(torch.tensor(X, dtype=torch.float))
    samples = logit_dist.sample(torch.Size((256,))).exp()
    return (samples / samples.sum(-2, keepdim=True)).mean(0)

# inverse transform 
def inverse_transform(val, mean, std):
    return std * val + mean

In [14]:
# search for best point
n_restarts = 50
# x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = np.c_[x_mins, x_maxs]
search_grid = np.random.uniform(x_bounds[:, 0], x_bounds[:, 1], size=(10*n_restarts, len(x_bounds)))

mo_models = [reg_model1, reg_model2, reg_model3, cls_model] # includes all necessary models for optimization
mo_likls = [reg_likl1, reg_likl2, reg_likl3, cls_likl]

# Swith on eval mode
for i in range(len(mo_models)):
    mo_models[i].eval()
    mo_likls[i].eval()

y_best_curr = torch.max((train_y1 + train_y2 + train_y3)).item()

acf_vals = [acquisition_func(search_grid[i, :].reshape(1, -1), mo_models, mo_likls, y_best_curr) for i in range(10*n_restarts)]

acf_vals = np.array(acf_vals).reshape(10*n_restarts,)
top_idx = np.argsort(acf_vals)
search_grid = search_grid[top_idx[0:n_restarts]].reshape(n_restarts,-1)

best_acquisition_values = 1
best_x = None
for i, starting_point in enumerate(search_grid):
    res = minimize(fun=acquisition_func, 
                   x0=starting_point, 
                   bounds=x_bounds,
                   method='L-BFGS-B',
                   args=(mo_models, mo_likls, y_best_curr))
    if res.fun < best_acquisition_values:
        best_acquisition_values = res.fun
        best_x = res.x
    #print every 10 starting points
    if i%10 == 0:
        print('Iteration {0:d} - Best Acf value = {1:.3f}'.format(i, best_acquisition_values))



Iteration 0 - Best Acf value = -0.049
Iteration 10 - Best Acf value = -0.049
Iteration 20 - Best Acf value = -0.049
Iteration 30 - Best Acf value = -0.049
Iteration 40 - Best Acf value = -0.049


In [15]:
new_point = best_x
new_acf = (-1.0) * best_acquisition_values

In [16]:
def new_output(X, models, likelihoods, transsform_func, scaling):
    '''
    X: (np array) new best point
    models: (list) all model objects ; regression first and last model is classification
    likelihoods: (list) likelihood objects; last one is classification
    transform_func: (function handle) function to inverse transform all objective values
    scaling: (dictionary) values of x and y means and standard deviation used for scaling
    '''
    X = torch.tensor(X.reshape(1,-1), dtype=torch.float)
    obj_models = models[0:-1]
    class_model = models[-1]
    # value of objective function - inverse transformed
    obj_likls = likelihoods[0:-1]
    obj_preds = []
    for i in range(len(obj_models)):
        pred, _ = predict(obj_models[i], obj_likls[i], X)
        obj_preds.append(transsform_func(pred, scaling['ymeans'].values[i], scaling['ystddv'].values[i])) 
    # feasibility
    logit_dist = class_model(X)
    samples = logit_dist.sample(torch.Size((256,))).exp()
    class_probabilities = (samples / samples.sum(-2, keepdim=True)).mean(0)
    feasibility = class_probabilities[1] # Pr(feasible|X)
    #inverse transformed input
    X_tr = [transsform_func(X[0][i].item(), scaling['xmeans'].values[i], scaling['xstddv'].values[i]) for i in range(8)]

    return X_tr, obj_preds, feasibility

    

In [17]:
new_pt_tr, new_obj, new_feasib = new_output(new_point, mo_models, mo_likls, inverse_transform, scaling)

In [18]:
np.array(new_pt_tr)

array([ 5235.69939167,    65.04736558,    24.80448022, 13501.47653811,
          91.62501695,    76.91168355,    63.28927553,    99.5972619 ])

In [19]:
new_obj

[tensor([171.7722]), tensor([35.4881]), tensor([11.7783])]

In [20]:
#predict shear modulus -
reg_model3.eval()
reg_likl3.eval()
y3_pred, _ = predict(reg_model3, reg_likl3, torch.tensor(new_point.reshape(1,-1), dtype=torch.float))
inverse_transform(y3_pred, ymeans.values[2], ystddv.values[2])

tensor([11.7783])

In [21]:
new_feasib

tensor([0.9459])